In [1]:
import tspredict
import pandas as pd
import numpy as np
import os
import time
from keras.models import Sequential
from keras.models import model_from_json

Using TensorFlow backend.


Load trained LSTM model. Model was trained on the first 6 dates in the 2017 growing season with <15% cloud cover. Features included blue, green, NDVI bands. 

In [2]:
# load json and create model
json_file = open("/home/ec2-user/model_labeled.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

In [3]:
# load weights into model
model.load_weights("/home/ec2-user/model_labeled.h5")
print("Loaded model from disk")

Loaded model from disk


In [4]:
# mean and standard deviation of features from training data - used to standardize features for model prediction
mu = np.load('/home/ec2-user/mu.npy')
sd = np.load('/home/ec2-user/sd.npy')

In [5]:
# File paths to Sentinel-2 tiles to predict (intersecting the Rukwa region)
fp = '/home/ec2-user/sent-scenes-s3'
tiles = [fp + '/' + t for t in os.listdir(fp)]

In [6]:
for tile in tiles:
    print(tile)

/home/ec2-user/sent-scenes-s3/T36MTS
/home/ec2-user/sent-scenes-s3/T35LRL
/home/ec2-user/sent-scenes-s3/T35MRM
/home/ec2-user/sent-scenes-s3/T36MUT
/home/ec2-user/sent-scenes-s3/T36LTR
/home/ec2-user/sent-scenes-s3/T36MTT
/home/ec2-user/sent-scenes-s3/T35MRN
/home/ec2-user/sent-scenes-s3/T36MVS
/home/ec2-user/sent-scenes-s3/T36MUS
/home/ec2-user/sent-scenes-s3/T36LUR


In [ ]:
start_time = time.time()

for tile in tiles:
    # Reshape bands in each scene to match input shape required by Keras sequential model
    formatted_scene = tspredict.format_scene(tile, mu, sd)
    
    print(tile + ' formatted')
    
    # refimg can be any band from the same Sentinel-2 tile, outimg for writing pred. scene to disk
    band_paths = []
    for path, subdirs, files in os.walk(tile):
        for name in files:
            band_paths.append(os.path.join(path, name))
    
    refimg = band_paths[0]
    outimg = tile + '/tile_predicted.tif'
    
    # Predict the full tile
    predicted_tile = tspredict.classify_scene(formatted_scene=formatted_scene, model=model, 
                                              refimg=refimg, outimg=outimg)
    print(tile + ' predicted')

print("--- %s seconds ---" % (time.time() - start_time))

/home/ec2-user/sent-scenes-s3/T36MTS formatted
/home/ec2-user/sent-scenes-s3/T36MTS predicted


In [ ]:
# from joblib import Parallel, delayed
# import multiprocessing

# def predict_tile(tile):
#     # Reshape bands in each scene to match input shape required by Keras sequential model
#     formatted_scene = tspredict.format_scene(tile, mu, sd)
    
#      # refimg can be any band from the same Sentinel-2 tile, outimg for writing pred. scene to disk
#     band_paths = []
#     for path, subdirs, files in os.walk(tile):
#         for name in files:
#             band_paths.append(os.path.join(path, name))
    
#     refimg = band_paths[0]
#     outimg = tile + '/tile_predicted.tif'
    
#     # Predict the full tile
#     predicted_tile = tspredict.classify_scene(formatted_scene=formatted_scene, model=model, 
#                                               refimg=refimg, outimg=outimg)

# # Perform computation in parallel
# #Parallel(n_jobs=-1, backend="multiprocessing")(map(delayed(predict_tile), tiles))
# Parallel(n_jobs=-1)(delayed(predict_tile)(tile) for tile in tiles)